In [ ]:
import cv2
import sys
import pandas as pd



In [ ]:
video_path=r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs preprocess\JOMR_nov25_BSD_01.mp4'
play_speed=1.0
team1_name="JOMR"
team2_name="MyH_HeR"
output_dir= r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\points_segments'

temp_list=cv2_point_segment_cut(
    video_path=video_path,
    play_speed=play_speed,
    team1_name=team1_name,
    team2_name=team2_name,
    output_dir=output_dir
)

In [ ]:
# from montage_auto_frames_gpu import cv2_temp_list_to_operate
import pandas as pd
import cv2
import sys

def cv2_point_segment_cut(
        video_path : str,
        play_speed : float = 1.0,
        team1_name: str = "JOMR",
        team2_name: str = "adversaire",
        output_dir: str = None
    ) -> pd.DataFrame:

    """
    Création d'un dataframe contenant les segments de points extraits d'une vidéo de match, avec les informations sur le score et les équipes.
    A utiliser ensuite avec cut_point_gpu pour découper les segments de points à partir de ce dataframe. 
    Permet également de contrôler la lecture de la vidéo (pause, vitesse) pour faciliter l'identification des points.
    
    Args:
        video_path (str): Chemin vers la vidéo à traiter.
        play_speed (float, optional): Vitesse de lecture de la vidéo (1=normale, 0=pause, >1=plus rapide). Par défaut à 1.0.
        team1_name (str, optional): Nom de l'équipe 1. Par défaut à "JOMR".
        team2_name (str, optional): Nom de l'équipe 2. Par défaut à "adversaire".
        output_dir (str, optional): Dossier de sortie pour les segments extraits. Si None, les segments seront extraits dans le même dossier que la vidéo source.
    Returns:
        DataFrame contenant les informations sur les segments de points extraits, avec les colonnes : 
        'point_index',
        'action',
        'start_frame',
        'end_frame',
        'score_team1',
        'score_team2',
        'set_team1',
        'set_team2'
    """
    # Initialisation des variables
    temp_list = list()
    last_action = None

    # Mapping des touches aux temp_list
    key_action_map = {
        ord('0'): 'debut du set',
        ord('1'): f'service {team1_name}',
        ord('3'): f'service {team2_name}',
        ord('2'): 'fin point'
        }


    # Afficher les touches disponibles en overlay sur la vidéo
    help_lines = [
        "0 : debut du set",
        f"1 : service {team1_name}",
        f"3 : service {team2_name}",
        "2 : fin du point"
    ]



    _orig_imshow = cv2.imshow

    def _imshow_with_help(winname, frame):
        if frame is not None:
            x, y = 30, 120
            for i, line in enumerate(help_lines):
                cv2.putText(frame,
                            line,
                            (x, y + i * 25),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.7,
                            (255, 255, 255),
                            2,
                            cv2.LINE_AA)
        _orig_imshow(winname, frame)

    cv2.imshow = _imshow_with_help


    # Ouvrir la vidéo
    cap = cv2.VideoCapture(video_path)

    def _waitKey_fast(ms):
        # Réduire le délai proportionnellement à la vitesse (au moins 1 ms)
        adj = max(1, int(ms / play_speed))
        return cv2.waitKey(adj)

    # Vérifier que la vidéo est bien ouverte
    if not cap.isOpened():
        print("Erreur : impossible d’ouvrir la vidéo.")
        sys.exit()

    # Récupérer les FPS pour convertir les frames en temps
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_number = 0

    # État de pause et rotation
    paused = False


    try:
        while cap.isOpened():

            # Lire une frame seulement si on n'est pas en pause
            if not paused:
                ret, frame = cap.read()

                # Arrêter si fin de vidéo ou erreur
                if not ret:
                    print("Fin de la vidéo ou erreur de lecture.")
                    break

                # Incrémenter le compteur de frames
                frame_number += 1

            # Affiche la dernière action
            if last_action and ret:
                cv2.putText(frame,
                            f"Derniere action : {last_action}",
                            (30, 40),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 255, 0),
                            2,
                            cv2.LINE_AA)

            # Indiquer le mode pause sur l'image affichée
            if paused and ret:
                cv2.putText(frame,
                            "|| PAUSE ||",
                            (30, 80),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 0, 255),
                            2,
                            cv2.LINE_AA)

            # Afficher la frame courante
            if ret:
                cv2.imshow(f'{video_path}', frame)

            # Gestion des entrées clavier
            key = _waitKey_fast(30) & 0xFF
            if key == ord('q'):  # quitter
                break
            elif key == ord(' '):  # pause/reprise
                paused = not paused
            elif key == ord('+'):  # augmenter la vitesse
                play_speed += 0.5
                continue
            elif key == ord('-'):  # diminuer la vitesse
                play_speed = max(0.5, play_speed - 0.5)
                continue
            elif key in key_action_map: # enregistrer l'action associée à la touche, avec le numéro de frame
                action_name = key_action_map[key]
                last_action = action_name

                temp_list.append({
                    'Frame': frame_number,
                    'Action': action_name
                })


    finally:
        # Libérer les ressources OpenCV
        cap.release()
        cv2.destroyAllWindows()


    # print(temp_list)

    # Initialiser une liste pour stocker les segments de points
    list_actions = list()

    i = 0
    while i < len(temp_list):
        action = temp_list[i]
        

        # Si l'action n'est pas 'fin point', on la traite
        if action['Action'] != 'fin point':
            start_frame = action['Frame']
            service_side = action['Action']
            # Chercher le prochain 'fin point'
            end_frame = None
            j = i + 1
            while j < len(temp_list):
                if temp_list[j]['Action'] == 'fin point':
                    end_frame = temp_list[j]['Frame']
                    break
                j += 1
            
            # Ajouter à la liste si on a trouvé un 'fin point'
            if end_frame is not None:
                list_actions.append({
                    'Service_side': service_side,
                    'Start_frame': start_frame,
                    'End_frame': end_frame
                })
        
        i += 1

    # Convertir en DataFrame pandas
    df_points = pd.DataFrame(list_actions)

    # Ajouter les colonnes de score
    df_points[f'{team1_name}_score'] = 0
    df_points[f'{team2_name}_score'] = 0
    df_points[f'{team1_name}_sets'] = 0
    df_points[f'{team2_name}_sets'] = 0


    # Mettre à jour les scores en fonction du service_side
    for idx, row in df_points.iterrows():
            if df_points['Service_side'].iloc[idx] == 'debut du set':
                # Première ligne : initialiser selon le service
                if row['Service_side'] == f'service {team1_name}':
                    df_points.at[idx, f'{team1_name}_score'] = 1
                    df_points.at[idx, f'{team2_name}_score'] = 0
                elif row['Service_side'] == f'service {team2_name}':
                    df_points.at[idx, f'{team1_name}_score'] = 0
                    df_points.at[idx, f'{team2_name}_score'] = 1
            else:
                # Lignes suivantes : reprendre le score précédent et incrémenter selon le service
                df_points.at[idx, f'{team1_name}_score'] = df_points.at[idx - 1, f'{team1_name}_score']
                df_points.at[idx, f'{team2_name}_score'] = df_points.at[idx - 1, f'{team2_name}_score']
                
                if row['Service_side'] == f'service {team1_name}':
                    df_points.at[idx, f'{team1_name}_score'] += 1
                elif row['Service_side'] == f'service {team2_name}':
                    df_points.at[idx, f'{team2_name}_score'] += 1

    # Mettre à jour les scores de sets au début de chaque nouveau set
    for idx, row in df_points.iterrows():
        if row['Service_side'] == 'debut du set' and idx > 0:
            # Comparer les scores du set précédent
            prev_score_team1 = df_points.at[idx - 1, f'{team1_name}_score']
            prev_score_team2 = df_points.at[idx - 1, f'{team2_name}_score']
            
            # Récupérer les sets précédents
            df_points.at[idx, f'{team1_name}_sets'] = df_points.at[idx - 1, f'{team1_name}_sets']
            df_points.at[idx, f'{team2_name}_sets'] = df_points.at[idx - 1, f'{team2_name}_sets']
            
            # Ajouter un set au gagnant
            if prev_score_team1 > prev_score_team2:
                df_points.at[idx, f'{team1_name}_sets'] += 1
            else:
                df_points.at[idx, f'{team2_name}_sets'] += 1
        elif idx > 0:
            # Pour les autres lignes, conserver le nombre de sets
            df_points.at[idx, f'{team1_name}_sets'] = df_points.at[idx - 1, f'{team1_name}_sets']
            df_points.at[idx, f'{team2_name}_sets'] = df_points.at[idx - 1, f'{team2_name}_sets']

    return df_points

# -------------------------------------------------------------------------------------------------------------




In [58]:
test_df_points = cv2_point_segment_cut(
    video_path=r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs preprocess\JOMR_nov25_BSD_01.mp4',
)

test_df_points

,Service_side,Start_frame,End_frame,JOMR_score,adversaire_score,JOMR_sets,adversaire_sets
0,debut du set,38,190,0,0,0,0
1,service JOMR,406,681,1,0,0,0
2,service adversaire,1190,1422,1,1,0,0
3,service JOMR,1702,2003,2,1,0,0
4,service adversaire,2441,2783,2,2,0,0
5,debut du set,3254,4223,0,0,0,1
6,service JOMR,3683,4223,1,0,0,1
7,service JOMR,4526,5025,2,0,0,1
8,service JOMR,5105,5590,3,0,0,1


In [50]:
temp_list = [{'Frame': 45, 'Action': 'debut du set'}, {'Frame': 186, 'Action': 'fin point'}, {'Frame': 431, 'Action': 'service JOMR'}, {'Frame': 690, 'Action': 'fin point'}, {'Frame': 1140, 'Action': 'service adversaire'}, {'Frame': 1493, 'Action': 'fin point'}, {'Frame': 1703, 'Action': 'service JOMR'}, {'Frame': 2396, 'Action': 'fin point'}, {'Frame': 2440, 'Action': 'JOMR'}, {'Frame': 2604, 'Action': 'fin point'},{'Frame': 45, 'Action': 'debut du set'}, {'Frame': 186, 'Action': 'fin point'}, {'Frame': 431, 'Action': 'service JOMR'}, {'Frame': 690, 'Action': 'fin point'}, {'Frame': 1140, 'Action': 'service adversaire'}, {'Frame': 1493, 'Action': 'fin point'}, {'Frame': 1703, 'Action': 'service JOMR'}, {'Frame': 2396, 'Action': 'fin point'}, {'Frame': 2440, 'Action': 'JOMR'}, {'Frame': 2604, 'Action': 'fin point'}]
temp_list

[{'Frame': 45, 'Action': 'debut du set'},
 {'Frame': 186, 'Action': 'fin point'},
 {'Frame': 431, 'Action': 'service JOMR'},
 {'Frame': 690, 'Action': 'fin point'},
 {'Frame': 1140, 'Action': 'service adversaire'},
 {'Frame': 1493, 'Action': 'fin point'},
 {'Frame': 1703, 'Action': 'service JOMR'},
 {'Frame': 2396, 'Action': 'fin point'},
 {'Frame': 2440, 'Action': 'JOMR'},
 {'Frame': 2604, 'Action': 'fin point'},
 {'Frame': 45, 'Action': 'debut du set'},
 {'Frame': 186, 'Action': 'fin point'},
 {'Frame': 431, 'Action': 'service JOMR'},
 {'Frame': 690, 'Action': 'fin point'},
 {'Frame': 1140, 'Action': 'service adversaire'},
 {'Frame': 1493, 'Action': 'fin point'},
 {'Frame': 1703, 'Action': 'service JOMR'},
 {'Frame': 2396, 'Action': 'fin point'},
 {'Frame': 2440, 'Action': 'JOMR'},
 {'Frame': 2604, 'Action': 'fin point'}]

In [ ]:
import pandas as pd

team1_name=str("JOMR")
team2_name=str("adversaire")


# Créer le dataframe à partir de 'temp_list'

# Initialiser une liste pour stocker les segments de points
list_actions = list()

i = 0
while i < len(temp_list):
    action = temp_list[i]
    

    # Si l'action n'est pas 'fin point', on la traite
    if action['Action'] != 'fin point':
        start_frame = action['Frame']
        service_side = action['Action']
        # Chercher le prochain 'fin point'
        end_frame = None
        j = i + 1
        while j < len(temp_list):
            if temp_list[j]['Action'] == 'fin point':
                end_frame = temp_list[j]['Frame']
                break
            j += 1
        
        # Ajouter à la liste si on a trouvé un 'fin point'
        if end_frame is not None:
            list_actions.append({
                'Service_side': service_side,
                'Start_frame': start_frame,
                'End_frame': end_frame
            })
    
    i += 1

# Convertir en DataFrame pandas
df_points = pd.DataFrame(list_actions)

# Ajouter les colonnes de score
df_points[f'{team1_name}_score'] = 0
df_points[f'{team2_name}_score'] = 0
df_points[f'{team1_name}_sets'] = 0
df_points[f'{team2_name}_sets'] = 0


# Mettre à jour les scores en fonction du service_side
for idx, row in df_points.iterrows():
        if df_points['Service_side'].iloc[idx] == 'debut du set':
            # Première ligne : initialiser selon le service
            if row['Service_side'] == f'service {team1_name}':
                df_points.at[idx, f'{team1_name}_score'] = 1
                df_points.at[idx, f'{team2_name}_score'] = 0
            elif row['Service_side'] == f'service {team2_name}':
                df_points.at[idx, f'{team1_name}_score'] = 0
                df_points.at[idx, f'{team2_name}_score'] = 1
        else:
            # Lignes suivantes : reprendre le score précédent et incrémenter selon le service
            df_points.at[idx, f'{team1_name}_score'] = df_points.at[idx - 1, f'{team1_name}_score']
            df_points.at[idx, f'{team2_name}_score'] = df_points.at[idx - 1, f'{team2_name}_score']
            
            if row['Service_side'] == f'service {team1_name}':
                df_points.at[idx, f'{team1_name}_score'] += 1
            elif row['Service_side'] == f'service {team2_name}':
                df_points.at[idx, f'{team2_name}_score'] += 1

# Mettre à jour les scores de sets au début de chaque nouveau set
for idx, row in df_points.iterrows():
    if row['Service_side'] == 'debut du set' and idx > 0:
        # Comparer les scores du set précédent
        prev_score_team1 = df_points.at[idx - 1, f'{team1_name}_score']
        prev_score_team2 = df_points.at[idx - 1, f'{team2_name}_score']
        
        # Récupérer les sets précédents
        df_points.at[idx, f'{team1_name}_sets'] = df_points.at[idx - 1, f'{team1_name}_sets']
        df_points.at[idx, f'{team2_name}_sets'] = df_points.at[idx - 1, f'{team2_name}_sets']
        
        # Ajouter un set au gagnant
        if prev_score_team1 > prev_score_team2:
            df_points.at[idx, f'{team1_name}_sets'] += 1
        else:
            df_points.at[idx, f'{team2_name}_sets'] += 1
    elif idx > 0:
        # Pour les autres lignes, conserver le nombre de sets
        df_points.at[idx, f'{team1_name}_sets'] = df_points.at[idx - 1, f'{team1_name}_sets']
        df_points.at[idx, f'{team2_name}_sets'] = df_points.at[idx - 1, f'{team2_name}_sets']


print(list_actions)
df_points

[{'Service_side': 'debut du set', 'Start_frame': 45, 'End_frame': 186}, {'Service_side': 'service JOMR', 'Start_frame': 431, 'End_frame': 690}, {'Service_side': 'service adversaire', 'Start_frame': 1140, 'End_frame': 1493}, {'Service_side': 'service JOMR', 'Start_frame': 1703, 'End_frame': 2396}, {'Service_side': 'JOMR', 'Start_frame': 2440, 'End_frame': 2604}, {'Service_side': 'debut du set', 'Start_frame': 45, 'End_frame': 186}, {'Service_side': 'service JOMR', 'Start_frame': 431, 'End_frame': 690}, {'Service_side': 'service adversaire', 'Start_frame': 1140, 'End_frame': 1493}, {'Service_side': 'service JOMR', 'Start_frame': 1703, 'End_frame': 2396}, {'Service_side': 'JOMR', 'Start_frame': 2440, 'End_frame': 2604}]


,Service_side,Start_frame,End_frame,JOMR_score,adversaire_score,JOMR_sets,adversaire_sets
0,debut du set,45,186,0,0,0,0
1,service JOMR,431,690,1,0,0,0
2,service adversaire,1140,1493,1,1,0,0
3,service JOMR,1703,2396,2,1,0,0
4,JOMR,2440,2604,2,1,0,0
5,debut du set,45,186,0,0,1,0
6,service JOMR,431,690,1,0,1,0
7,service adversaire,1140,1493,1,1,1,0
8,service JOMR,1703,2396,2,1,1,0
9,JOMR,2440,2604,2,1,1,0
